In [1]:
import torch
import torchvision
from torchvision.models import alexnet
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
import numpy as np
from torch.autograd import Variable
from scipy.misc import imresize
import torch.cuda
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

 Задание: выкинуть последний слой из vgg / alexnet и обучить линейную регрессиию на выходах полученных сетей.
Например, для mnist.

In [28]:
anet = torchvision.models.alexnet(pretrained = True)
fmodel = anet.features

In [29]:
for m in fmodel.children():
    for p in m.parameters():
        p.requires_grad = False

In [30]:
fmodel.cuda()

Sequential (
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU (inplace)
  (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU (inplace)
  (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU (inplace)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU (inplace)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU (inplace)
  (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
)

In [31]:
mnist = fetch_mldata('MNIST original', data_home='~/.datasets')
images = mnist['data']
images = images.reshape(images.shape[0], 1, 28, 28) # data dims: examples x channels x w x h
images = images.astype(np.float32) / 255 # float and normalize
labels = mnist['target'].astype(int)
np_images_train, np_images_test, np_labels_train, np_labels_test = train_test_split(images, labels, train_size = 0.9)

In [32]:
def resize(batch):
    factor = 8
    resized_images = np.empty((batch.shape[0], 1, batch.shape[2] * 8, batch.shape[3] * 8))
    for i in xrange(batch.shape[0]):
        resized_images[i, 0, :] = imresize(batch[i, 0], size = float(factor))
    out_images = np.concatenate([resized_images] * 3, axis = 1)
    return out_images

In [37]:
def learn(data, labels, iters = None):
    if iters is None:
        iters = int(np.ceil(float(data.shape[0]) / batchsize))
    batchsize = 64
    res = np.empty((batchsize * iters, 256 * 6 * 6))
    res_labels = labels[:batchsize * iters]
    for it in tqdm(xrange(iters)):
        orig_batch = data[it * batchsize:(it + 1) * batchsize]
        batch = resize(orig_batch)
        invar = Variable(torch.from_numpy(batch)).float().cuda()
        outvar = fmodel.forward(invar)
        flat = outvar.view(outvar.size(0), -1)
        flatres = flat.data.cpu().numpy()
        res[it * batchsize:(it + 1) * batchsize] = flatres
    return res, res_labels

In [8]:
# batchsize = 64
# # iters = int(np.ceil(float(np_images_train.shape[0]) / batchsize))
# iters = 200 # my system freezes after 250 iterations
# # res = np.empty((np_images_train.shape[0], 256 * 6 * 6))
# res = np.empty((batchsize * iters, 256 * 6 * 6))
# res_labels = np_labels_train[:batchsize * iters]

In [12]:
# for it in tqdm(xrange(iters)):
#     orig_batch = np_images_train[it * batchsize:(it + 1) * batchsize]
#     batch = resize(orig_batch)
#     invar = Variable(torch.from_numpy(batch)).float().cuda()
#     outvar = fmodel.forward(invar)
#     flat = outvar.view(outvar.size(0), -1)
#     flatres = flat.data.cpu().numpy()
#     res[it * batchsize:(it + 1) * batchsize] = flatres

100%|██████████| 200/200 [01:12<00:00,  2.85it/s]


In [34]:
iters = 200
res, res_labels = learn(np_images_train, np_labels_train, iters)

100%|██████████| 5/5 [00:01<00:00,  2.86it/s]


In [15]:
np.save('res', res)
np.save('labels', res_labels)

In [3]:
res = np.load('res.npy')
res_labels = np.load('labels.npy')

In [8]:
lin = LinearRegression()

In [9]:
lin.fit(res, res_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [16]:
np.save('coef', lin.coef_)

In [10]:
batchsize = 64
# iters = int(np.ceil(float(np_images_train.shape[0]) / batchsize))
iters = 10 # my system freezes after 250 iterations
test_res = np.empty((batchsize * iters, 256 * 6 * 6))

In [38]:
test_res, test_res_labels = learn(np_images_test, np_labels_test, 20)

100%|██████████| 20/20 [00:06<00:00,  2.88it/s]


In [39]:
test_pred = lin.predict(test_res)

In [51]:
diff = np.abs(test_pred - test_res_labels)

In [52]:
diff[:100]

array([  4.21667351e-01,   6.56489946e-02,   3.49648696e-01,
         1.72779428e+00,   3.22825962e-01,   4.95604712e-01,
         1.31029449e+00,   3.78463356e-01,   4.52699546e-01,
         9.04841639e-01,   5.22935792e+00,   2.65573334e+00,
         2.05713091e+00,   4.86902771e-01,   1.29841991e+00,
         1.57717807e+00,   1.17124604e+00,   6.65168989e-01,
         1.19868264e+00,   1.24911629e+00,   7.52137807e-01,
         1.58027875e+00,   2.59414641e-01,   3.39754800e-01,
         1.42578161e+00,   3.20700847e-01,   4.21773120e-01,
         5.38389632e-01,   8.76520049e-01,   5.14307258e-01,
         1.94820709e-01,   3.01659026e+00,   8.90554584e+00,
         6.49866499e-02,   3.47280866e-02,   4.32850455e-01,
         2.29313225e-01,   1.15484801e-01,   3.06592109e-01,
         2.44375731e+00,   3.75470192e-01,   1.53411957e+00,
         1.43419186e+00,   1.14382652e+00,   2.48355459e+00,
         9.16597901e-01,   1.01969478e+00,   8.64421820e-01,
         3.50757145e-01,